In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/reversible2/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible.sliced import sliced_from_samples

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
from reversible.plot import create_bw_image
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible.revnet import ResidualBlock, invert, SubsampleSplitter, ViewAs, ReversibleBlockOld
from spectral_norm import spectral_norm
from conv_spectral_norm import conv_spectral_norm

def display_text(text, fontsize=18):
    fig = plt.figure(figsize=(12,0.1))
    plt.title(text, fontsize=fontsize)
    plt.axis('off')
    display(fig)
    plt.close(fig)

In [ ]:
import pickle
import gzip
## from  http://deeplearning.net/data/mnist/mnist.pkl.gz
train, val, test = pickle.load(gzip.open('/data/schirrmr/schirrmr/mnist/mnist.pkl.gz'), encoding='bytes')

X_train, y_train = train
X_val, y_val = val

X_train_topo = X_train.reshape(X_train.shape[0], 1, 28,28)
X_val_topo = X_val.reshape(X_val.shape[0], 1, 28,28)
from numpy.random import RandomState

In [ ]:
from reversible.util import np_to_var, var_to_np

In [ ]:
x_0 = X_train_topo[y_train == 0][:50]
x_1 = X_train_topo[y_train == 1][:50]

In [ ]:
inputs = [np_to_var(x_0, dtype=np.float32).cuda(),
          np_to_var(x_1, dtype=np.float32).cuda()]

In [ ]:
for i_class in range(2):
    image_grid = np.zeros((5,10,28,28))
    for i_row in range(image_grid.shape[0]):
        for i_col in range(image_grid.shape[1]):
            image_grid[i_row, i_col] = var_to_np(inputs[i_class][i_row * image_grid.shape[1] + i_col])
    im = create_bw_image(image_grid).resize((image_grid.shape[1]*100,image_grid.shape[0]*100))
    display(im)

In [ ]:
from reversible.revnet import ReversibleBlockOld
import torch.nn as nn

def rev_block(n_chans, n_intermediate_chans, kernel_size=3):
    c = n_chans // 2
    n_i_c = n_intermediate_chans
    assert kernel_size % 2 == 1
    padding = kernel_size // 2
    return ReversibleBlockOld(
        nn.Sequential(
            nn.Conv2d(c, n_i_c, (kernel_size, kernel_size), padding=padding),
             nn.ReLU(),
             nn.Conv2d(n_i_c, c, (kernel_size,kernel_size), padding=padding)),
        nn.Sequential(
            nn.Conv2d(c, n_i_c, (kernel_size,kernel_size), padding=padding),
            nn.ReLU(),
            nn.Conv2d(n_i_c, c, (kernel_size,kernel_size), padding=padding)))
def res_block(n_c, n_i_c):
     return ResidualBlock(
        nn.Sequential(
        nn.Conv2d(n_c, n_i_c, (3,3), stride=1, padding=(1,1),bias=True),
        nn.ReLU(),
            nn.Conv2d(n_i_c, n_c, (3,3), stride=1, padding=(1,1),bias=True)),
    )

In [ ]:
from reversible.revnet import SubsampleSplitter, ViewAs
from reversible.util import set_random_seeds
from reversible.revnet import init_model_params
from torch.nn import ConstantPad2d
import torch as th
set_random_seeds(3049, True)
feature_model = th.nn.Sequential(
    ConstantPad2d((2,2,2,2), 0),
    SubsampleSplitter(stride=2,checkerboard=True, chunk_chans_first=False),
    rev_block(4,25),
    rev_block(4,25),
    SubsampleSplitter(stride=2,checkerboard=True),
    rev_block(16,50),
    rev_block(16,50),
    SubsampleSplitter(stride=2,checkerboard=True),
    rev_block(64,100),
    rev_block(64,100),
    SubsampleSplitter(stride=2,checkerboard=True),
    rev_block(256,200),
    rev_block(256,200),
    SubsampleSplitter(stride=2,checkerboard=True),
    rev_block(1024,200, kernel_size=1),
    ViewAs((-1,1024,1,1),(-1,1024)),)

feature_model = feature_model.cuda()

from discriminator import ProjectionDiscriminator

from reversible.training import hard_init_std_mean
n_dims = 1024
n_clusters = 2
means_per_cluster = [th.autograd.Variable(th.zeros(n_dims).cuda(), requires_grad=True)
                     for _ in range(n_clusters)]


stds_per_cluster = [th.autograd.Variable(th.zeros(n_dims).cuda(), requires_grad=True)
                    for _ in range(n_clusters)]

for i_class in range(2):
    this_outs = feature_model(inputs[i_class])
    means_per_cluster[i_class].data = th.mean(this_outs, dim=0).data
    stds_per_cluster[i_class].data = th.log(th.std(this_outs, dim=0)).data

from copy import deepcopy
optimizer = th.optim.Adam(
                          [
    {'params': list(feature_model.parameters()),
    'lr': 0.0001,
    'weight_decay': 0},], betas=(0,0.9))


adv_model = nn.Sequential(
    nn.Conv2d(1,16, (3,3), stride=1, padding=(1,1),bias=True),#28
    res_block(16,32),
    res_block(16,32),
    nn.AvgPool2d((2,2)),#14
    res_block(16,32),
    res_block(16,32),
    nn.AvgPool2d((2,2)),#7
    res_block(16,32),
    res_block(16,32),
    nn.AvgPool2d((2,2), stride=(1,1)),#6
    res_block(16,32),
    res_block(16,32),
    nn.AvgPool2d((2,2)),
    ViewAs((-1,16,3,3), (-1,144)),
    )

adv_model = ProjectionDiscriminator(adv_model,144,2,)

adv_model.cuda()
        
optim_dist = th.optim.Adam(
                          [
    {'params': means_per_cluster + stds_per_cluster,
    'lr': 1e-2,
    'weight_decay': 0},], betas=(0,0.9))


optim_adv = th.optim.Adam([{
    'params': adv_model.parameters(),
    'lr': 4e-3, 'weight_decay': 0.00}],#lr 0.0004
                         betas=(0,0.9))


In [ ]:
from reversible.gaussian import get_gauss_samples
from reversible.uniform import get_uniform_samples
from reversible.revnet import invert 
import pandas as pd
from gradient_penalty import gradient_penalty
import time


df = pd.DataFrame()
g_loss = np_to_var([np.nan],dtype=np.float32)
g_grad = np.nan
d_loss = np_to_var([np.nan],dtype=np.float32)
d_grad = np.nan

In [ ]:

other_outs[:,500] = means_per_cluster[i_class][500].unsqueeze(0) + (
    th.randn(len(other_outs)).cuda() * stds_per_cluster[i_class][500])

In [ ]:
n_epochs = 10001
for i_epoch in range(n_epochs):
    start_time = time.time()
    optim_adv.zero_grad()
    optimizer.zero_grad()
    optim_dist.zero_grad()
    for i_class in range(len(inputs)):
        mean = means_per_cluster[i_class]
        log_std = stds_per_cluster[i_class]
        if i_class == 1:
            mean = th.cat((means_per_cluster[0][0:500], mean[500:501],
                         means_per_cluster[0][501:]))
            log_std = th.cat((stds_per_cluster[0][0:500], log_std[500:501],
                         stds_per_cluster[0][501:]))
        std = th.exp(log_std)
        this_inputs = inputs[i_class]
        y = np_to_var([i_class]).cuda()
        samples = get_gauss_samples(len(this_inputs), mean, std, truncate_to=3)
        other_outs = feature_model(inputs[i_class - 1])
        
        # move to this class
        other_outs[:,500] = means_per_cluster[i_class][500].unsqueeze(0) + (
            th.randn(len(other_outs)).cuda() * stds_per_cluster[i_class][500])
        samples = th.cat((samples, other_outs), dim=0)

        inverted = invert(feature_model, samples)
        score_fake = adv_model(inverted, y)
        if (i_epoch % 10) != 0:
            score_real = adv_model(this_inputs, y)
            gradient_loss = gradient_penalty(adv_model, this_inputs, inverted[:(len(this_inputs))], y)
            d_loss = -score_real.mean() + score_fake.mean() + gradient_loss * 10
            d_loss.backward()
            d_grad = np.mean([th.sum(p.grad **2).item() for p in adv_model.parameters()])
        else:
            g_loss = -th.mean(score_fake)
            g_loss.backward()
            g_grad = np.mean([th.sum(p.grad **2).item() for p in feature_model.parameters()])
    if (i_epoch % 10) != 0:
            optim_adv.step()
    else:
        optimizer.step()
        optim_dist.step()
    with th.no_grad():
        sample_wd_row = {}
        for i_class in range(len(inputs)):
            this_inputs = inputs[i_class]
            mean = means_per_cluster[i_class]
            log_std = stds_per_cluster[i_class]
            if i_class == 1:
                mean = th.cat((means_per_cluster[0][0:500], mean[500:501],
                             means_per_cluster[0][501:]))
                log_std = th.cat((stds_per_cluster[0][0:500], log_std[500:501],
                             stds_per_cluster[0][501:]))
            std = th.exp(log_std)
            y = np_to_var([i_class]).cuda()
            samples = get_gauss_samples(len(this_inputs), mean, std, truncate_to=3)

            inverted = invert(feature_model, samples)
            in_np = var_to_np(this_inputs).reshape(len(this_inputs), -1)
            fake_np = var_to_np(inverted).reshape(len(inverted), -1)
            import ot

            dist = np.sqrt(np.sum(np.square(in_np[:,None] - fake_np[None]), axis=2))
            match_matrix = ot.emd([],[], dist)
            cost = np.sum(dist * match_matrix)
            score_fake = adv_model(inverted, y)
            score_real = adv_model(inputs[i_class], y)
            wd_dist = th.mean(score_real) - th.mean(score_fake)
            sample_wd_row.update({
                'wd_dist_' + str(i_class): wd_dist.item() ,
                'sampled_wd' + str(i_class): cost,
                'wd_diff' + str(i_class): cost - wd_dist.item(),
            })
        end_time = time.time()
        epoch_row = {
        'd_loss': d_loss.item(),
        'g_loss': g_loss.item(),
        'o_real': th.mean(score_real).item(),
        'o_fake': th.mean(score_fake).item(),
        'g_grad': g_grad,
        'd_grad': d_grad,
        'runtime': end_time -start_time,}
        epoch_row.update(sample_wd_row)
        df = df.append(epoch_row, ignore_index=True)
        if i_epoch % (max(1,n_epochs // 100)) == 0:
            display_text("Epoch {:d}".format(i_epoch))
            display(df.iloc[-5:])
        if i_epoch % (n_epochs // 20) == 0:
            print("stds\n", var_to_np(th.exp(th.stack(stds_per_cluster))))
            for i_class in range(len(stds_per_cluster)):
                mean = means_per_cluster[i_class]
                log_std = stds_per_cluster[i_class]
                if i_class == 1:
                    mean = th.cat((means_per_cluster[0][0:500], mean[500:501],
                                 means_per_cluster[0][501:]))
                    log_std = th.cat((stds_per_cluster[0][0:500], log_std[500:501],
                                 stds_per_cluster[0][501:]))
                std = th.exp(log_std)
                y = np_to_var([i_class]).cuda()
                samples = get_gauss_samples(len(this_inputs), mean, std, truncate_to=3)
                inverted = invert(feature_model, samples).squeeze()
                image_grid = np.zeros((5,10,28,28))
                for i_row in range(image_grid.shape[0]):
                    for i_col in range(image_grid.shape[1]):
                        image_grid[i_row, i_col] = var_to_np(inverted[i_row * image_grid.shape[1] + i_col])
                im = create_bw_image(image_grid).resize((image_grid.shape[1]*50,image_grid.shape[0]*50))
                display(im)

            for i_class in range(len(inputs)):
                this_inputs = inputs[i_class]
                mean = means_per_cluster[i_class]
                log_std = stds_per_cluster[i_class]
                if i_class == 1:
                    mean = th.cat((means_per_cluster[0][0:500], mean[500:501],
                                 means_per_cluster[0][501:]))
                    log_std = th.cat((stds_per_cluster[0][0:500], log_std[500:501],
                                 stds_per_cluster[0][501:]))
                std = th.exp(log_std)
                y = np_to_var([i_class]).cuda()
                samples = get_gauss_samples(len(this_inputs) * 5, mean, std, truncate_to=3)

                inverted = invert(feature_model, samples)
                in_np = var_to_np(this_inputs).reshape(len(this_inputs), -1)
                fake_np = var_to_np(inverted).reshape(len(inverted), -1)
                import ot

                dist = np.sqrt(np.sum(np.square(in_np[:,None] - fake_np[None]), axis=2))
                match_matrix = ot.emd([],[], dist)
                cost = np.sum(dist * match_matrix)
                print("sliced", cost)
            a = means_per_cluster[0].detach()
            a = a.repeat(50,1)
            a.data[:,500] = th.linspace(means_per_cluster[0].data[500], means_per_cluster[1].data[500], a.shape[0])
            inverted = var_to_np(invert(feature_model, a).squeeze())
            display(create_bw_image(inverted.reshape(5,10,28,28)).resize((500,250)))
            for i_class in range(2):
                this_outs = feature_model(inputs[i_class],)

                this_outs.data[:,500] = means_per_cluster[1-i_class].data[500]

                inverted = var_to_np(invert(feature_model, this_outs).squeeze())

                image_grid = np.stack((var_to_np(inputs[i_class]).squeeze(), inverted), axis=0)
                display(create_bw_image(image_grid.swapaxes(0,1).reshape(10,10,28,28)).resize((500,500)))


In [ ]:
for i_class in range(2):
    this_outs = feature_model(inputs[i_class],)
    this_outs.data[:,500] = means_per_cluster[1-i_class].data[500]

    inverted = var_to_np(invert(feature_model, this_outs).squeeze())

    image_grid = np.stack((var_to_np(inputs[i_class]).squeeze(), inverted), axis=0)
    display(create_bw_image(image_grid.swapaxes(0,1).reshape(10,10,28,28)).resize((500,500)))


In [ ]:
for i_class in range(2):
    this_outs = feature_model(inputs[i_class],)
    this_outs.data[:,500] = means_per_cluster[1-i_class].data[500]

    inverted = var_to_np(invert(feature_model, this_outs).squeeze())
    angles_inv = np.array([angle_of_image(a) for a in inverted])
    angles_ins = np.array([angle_of_image(a) for a in var_to_np(inputs[i_class]).squeeze()])
angle_diffs = np.abs(angles_ins - angles_inv)

plt.figure(figsize=(3,3))
plt.plot(np.sin(angle_diffs), np.cos(angle_diffs), ls='', marker='o')
plt.ylim(-1.25,1.25)
plt.xlim(-0.25,1.25)
print(np.mean(angle_diffs))

In [ ]:
for i_class in range(2):
    this_outs = feature_model(inputs[i_class],)
    this_outs.data[:,500] = means_per_cluster[1-i_class].data[500]
    this_inputs = var_to_np(inputs[i_class].squeeze())
    inverted = var_to_np(invert(feature_model, this_outs).squeeze())
    angles_inv = np.array([angle_of_image(a) for a in inverted])
    angles_ins = np.array([angle_of_image(a) for a in this_inputs])
    for i_in in range(len(inverted)):
        im_pair = np.stack((this_inputs[i_in], inverted[i_in]), axis=0)
        display(create_bw_image(im_pair[None,:]).resize((88,44)))
        print(angles_ins[i_in], angles_inv[i_in])
    angle_diffs = np.abs(angles_ins - angles_inv)

    fig = plt.figure(figsize=(3,3))
    plt.plot(np.sin(angle_diffs), np.cos(angle_diffs), ls='', marker='o')
    plt.ylim(-1.25,1.25)
    plt.xlim(-0.25,1.25)
    display(fig)
    plt.close(fig)
    print(np.mean(angle_diffs))

In [ ]:
for i_class in range(2):
    this_inputs = var_to_np(inputs[i_class].squeeze())
    mean = means_per_cluster[1-i_class]
    log_std = stds_per_cluster[1-i_class]
    if i_class == 1:
        mean = th.cat((means_per_cluster[0][0:500], mean[500:501],
                     means_per_cluster[0][501:]))
        log_std = th.cat((stds_per_cluster[0][0:500], log_std[500:501],
                     stds_per_cluster[0][501:]))
    std = th.exp(log_std)
    y = np_to_var([i_class]).cuda()
    samples = get_gauss_samples(len(inputs[1-i_class]), mean, std, truncate_to=3)

    inverted = var_to_np(invert(feature_model, samples)).squeeze()
    angles_inv = np.array([angle_of_image(a) for a in inverted])
    angles_ins = np.array([angle_of_image(a) for a in this_inputs])
    for i_in in range(len(inverted)):
        im_pair = np.stack((this_inputs[i_in], inverted[i_in]), axis=0)
        display(create_bw_image(im_pair[None,:] > 0.7).resize((88,44)))
        print(angles_ins[i_in], angles_inv[i_in])
    angle_diffs = np.abs(angles_ins - angles_inv)

    fig = plt.figure(figsize=(3,3))
    plt.plot(np.sin(angle_diffs), np.cos(angle_diffs), ls='', marker='o')
    plt.ylim(-1.25,1.25)
    plt.xlim(-0.25,1.25)
    display(fig)
    plt.close(fig)
    print(np.mean(angle_diffs))

In [ ]:
for i_class in range(2):
    mean = means_per_cluster[1-i_class]
    log_std = stds_per_cluster[1-i_class]
    if i_class == 1:
        mean = th.cat((means_per_cluster[0][0:500], mean[500:501],
                     means_per_cluster[0][501:]))
        log_std = th.cat((stds_per_cluster[0][0:500], log_std[500:501],
                     stds_per_cluster[0][501:]))
    std = th.exp(log_std)
    y = np_to_var([i_class]).cuda()
    samples = get_gauss_samples(len(inputs[1-i_class]), mean, std, truncate_to=3)

    inverted = var_to_np(invert(feature_model, samples)).squeeze()
    angles_inv = np.array([angle_of_image(a) for a in inverted])
    angles_ins = np.array([angle_of_image(a) for a in var_to_np(inputs[i_class]).squeeze()])
    angle_diffs = np.abs(angles_ins - angles_inv)
    print(np.mean(angle_diffs))
    plt.figure(figsize=(3,3))
    plt.plot(np.sin(angle_diffs), np.cos(angle_diffs), ls='', marker='o')
    plt.ylim(-1.25,1.25)
    plt.xlim(-0.25,1.25)

In [ ]:
def angle_of_image(image):
    y,x = np.nonzero(image > 0.7)
    # now x = width, y = height
    y = 28 - y # to count height from bottom
    xy = np.stack((x,y)).T
    xy = xy -14
    pca = sklearn.decomposition.PCA().fit(xy)
    return (np.angle(pca.components_[0][0]+ pca.components_[0][1]*1j, deg=False)) % np.pi

In [ ]:
test_inputs = [np_to_var(X_train_topo[y_train == 0][50:], dtype=np.float32).cuda(),
      np_to_var(X_train_topo[y_train == 1][50:], dtype=np.float32).cuda()]
both_wanted_ims = []
for i_class in range(2):
    this_outs = feature_model(inputs[i_class],)

    plt.plot(var_to_np(this_outs)[:,500], var_to_np(this_outs)[:,500] * 0, ls='', marker='o',
            color=seaborn.color_palette()[i_class])
    test_outs = feature_model(test_inputs[i_class],)
    plt.plot(var_to_np(test_outs)[:,500], var_to_np(test_outs)[:,500] * 0, ls='', marker='x',
            color=seaborn.color_palette()[i_class])
    
    sorted_vals, i_sorted = th.sort(test_outs[:,500])
    wanted_inds = np.int64(np.round(np.linspace(0, len(i_sorted)-1,20)))
    wanted_inds = i_sorted[wanted_inds]

    wanted_ims = var_to_np(test_inputs[i_class][wanted_inds].squeeze())
    display(create_bw_image(wanted_ims[None]))
    both_wanted_ims.append(wanted_ims)

In [ ]:
create_bw_image(np.concatenate(np.array(both_wanted_ims)[::-1])[None]).resize((40*2*28,28*2))

In [ ]:
for i_class in range(2):
    this_outs = feature_model(inputs[i_class],)
    this_outs.data[:,500] = means_per_cluster[1-i_class].data[500]
    other_outs = feature_model(inputs[1-i_class])
    diffs = this_outs.unsqueeze(1) - other_outs.unsqueeze(0)
    diffs.data[:,:,500]= 0
    diffs = th.sum(diffs * diffs, dim=2)
    _, i_pairs = th.min(diffs,dim=1)

    inverted = var_to_np(invert(feature_model, this_outs).squeeze())
    image_grid = np.stack((var_to_np(inputs[i_class]).squeeze(), var_to_np(inputs[1-i_class][i_pairs]).squeeze()), axis=0)
    display(create_bw_image(image_grid.swapaxes(0,1).reshape(10,10,28,28)).resize((500,500)))

In [ ]:
for i_class in range(2):
    this_outs = feature_model(inputs[i_class],)

    this_outs.data[:,500] = means_per_cluster[1-i_class].data[500]

    inverted = var_to_np(invert(feature_model, this_outs).squeeze())

    image_grid = np.stack((var_to_np(inputs[i_class]).squeeze(), inverted), axis=0)
    display(create_bw_image(image_grid.swapaxes(0,1).reshape(10,10,28,28)).resize((500,500)))


In [ ]:
i_class = 0

In [ ]:
for i_class in range(2):
    this_outs = feature_model(inputs[i_class],)
    this_outs.data[:,500] = means_per_cluster[1-i_class].data[500]
    other_outs = feature_model(inputs[1-i_class])
    diffs = this_outs.unsqueeze(1) - other_outs.unsqueeze(0)
    diffs.data[:,:,500]= 0
    diffs = th.sum(diffs * diffs, dim=2)
    _, i_pairs = th.min(diffs,dim=1)

    image_grid = np.stack((var_to_np(inputs[i_class]).squeeze(), var_to_np(inputs[1-i_class][i_pairs]).squeeze()), axis=0)
    display(create_bw_image(image_grid.swapaxes(0,1).reshape(10,10,28,28)).resize((500,500)))
    
    
    angles_inv = np.array([angle_of_image(a) for a in var_to_np(inputs[i_class]).squeeze()])
    angles_ins = np.array([angle_of_image(a) for a in var_to_np(inputs[1-i_class][i_pairs]).squeeze()])
    angle_diffs = np.abs(angles_ins - angles_inv)
    print(np.mean(angle_diffs))
    plt.figure(figsize=(3,3))
    plt.plot(np.sin(angle_diffs), np.cos(angle_diffs), ls='', marker='o')
    plt.ylim(-1.25,1.25)
    plt.xlim(-0.25,1.25)
    
    angles_inv = np.array([angle_of_image(a) for a in var_to_np(inputs[i_class]).squeeze()])
    angles_ins = np.array([angle_of_image(a) for a in var_to_np(inputs[1-i_class][i_pairs[th.randperm(len(i_pairs))]]).squeeze()])
    angle_diffs = np.abs(angles_ins - angles_inv)
    print(np.mean(angle_diffs))
    print(np.corrcoef(np.mean(var_to_np(inputs[i_class].squeeze()), axis=(1,2)),
            np.mean(var_to_np(inputs[1-i_class][i_pairs]).squeeze(), axis=(1,2)),)[0,1])
    print(np.corrcoef(np.mean(var_to_np(inputs[i_class].squeeze()), axis=(1,2)),
            np.mean(var_to_np(inputs[1-i_class][i_pairs[th.randperm(len(i_pairs))]]).squeeze(), axis=(1,2)),)[0,1])

In [ ]:
#take a 0, move it to 1 see what it looks

In [ ]:
# also check accuracy of thresholded classifier on that dimension

In [ ]:
plt.plot(var_to_np(th.exp(stds_per_cluster[0])))

In [ ]:
th.exp(stds_per_cluster[1])[500]

In [ ]:
means_per_cluster[1][500]

In [ ]:
means_per_cluster[0][500]

In [ ]:
a = means_per_cluster[0].detach()

a = a.repeat(50,1)

a.data[:,500] = th.linspace(means_per_cluster[0].data[500], means_per_cluster[1].data[500], a.shape[0])

inverted = var_to_np(invert(feature_model, a).squeeze())

create_bw_image(inverted.reshape(5,10,28,28)).resize((1000,500))

In [ ]:
create_bw_image(inverted.reshape(5,10,28,28)).resize((1000,500))

In [ ]:
plt.plot(var_to_np(th.exp(stds_per_cluster[1])))